In [22]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.metrics import BinaryCrossentropy
from methods import *
import torch
import pickle, os
import skopt
from skopt import BayesSearchCV, gp_minimize
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
import warnings
warnings.filterwarnings("ignore")
print('done')

done


In [2]:
%load_ext autoreload
%autoreload 2

In [47]:
def get_predicted_label(model, image, device):
    output = model(image.to(device))
    softmax = nn.functional.softmax(out[0].cpu(), dim=0)
    pred_label = out.max(1)[1].item()
    return label

def save_obj(obj, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle)

def load_obj(filename):
    if not os.path.isfile(filename):
        print('Pickle {} does not exist.'.format(filename))
        return None
    with open(filename, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

def read_features(p_path, trigger_type_aux_str=None):
    """trigger_type_aux_str: used to select only rows from models with polygon or filter backdoors. If none, all data is chosen"""
    report = pd.read_csv(p_path)
    if trigger_type_aux_str is None:
        print('Using all rows (clean, polygon and instagram filters)')
    else:
        print(f'Using only clean rows and {trigger_type_aux_str}-backdoored rows from the dataset')
        indexes = []
        for i in range(len(report)):
            col = report['trigger_type_aux'].iloc[i]
            if (trigger_type_aux_str in col.lower()) or (col.lower() == 'none'):
                indexes.append(i)
        report = report.iloc[indexes]
    initial_columns = report.columns
    col_model_label = report['model_label'].copy(deep=True)
    for c in initial_columns:
        if not c.endswith('mean_diff') and not c.endswith('std_diff'):
            del report[c]
    features = report.values
    labels = np.array([int(col_model_label.iloc[i] == 'backdoor') for i in range(len(report))])
    return abs(features), labels

def read_features_confusion_matrix(p_path, trigger_type_aux_str=None):
    """trigger_type_aux_str: used to select only rows from models with polygon or filter backdoors. If none, all data is chosen"""
    report = pd.read_csv(p_path)
    initial_columns = report.columns
    col_model_label = report['model_label'].copy(deep=True)
    for c in initial_columns:
        if not c.startswith('h_') and not c.startswith('kl_'):
            del report[c]
    features = report.values
    labels = np.array([int(col_model_label.iloc[i] == 'backdoor') for i in range(len(report))])
    return abs(features), labels

def evaluate_classifier(train_x, train_y, test_x, test_y):
    #clf = svm.SVC(C=11, kernel='rbf', gamma='scale', probability=True)
    clf = LogisticRegression(C=2.0)
    clf.fit(train_x, train_y)
    y_score = clf.predict(test_x)
    y_pred = clf.predict_proba(test_x)
    roc_auc = roc_auc_score(y_true=test_y, y_score=y_score)
    cross_entropy = log_loss(y_true=test_y, y_pred=y_pred)
    return roc_auc, cross_entropy

def get_base_classifier():
    # return svm.SVC(C=11, kernel='rbf', gamma='scale', probability=True)
    return LogisticRegression()

print('done')

done


# Stratified K-fold validation for a full training dataset (square-size and 5 filters)

In [51]:
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square25-filters_triggered_classes.csv' # old dataset
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
n_splits = 10
n_repeats = 1
trigger_type_aux_str = None

for size in [30]: # [25, 30, 35, 40, 45, 50]:
    path_csv = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}-random-filters_all-classes_gray.csv'
    X, y = read_features(path_csv, trigger_type_aux_str) # clean data is automatically added
    
    pipeline = Pipeline([('standardize', StandardScaler()), ('LR', get_base_classifier())])
    kfold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=666)
    results = cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_log_loss', n_jobs=-1)
    print(f'CrossEntropy: mean={-results.mean()} std={results.std()}')

Using all rows (clean, polygon and instagram filters)
CrossEntropy: mean=0.5407713898374368 std=0.06386917641348638


# HPO for training data

In [22]:
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
path_csv = r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random-filters_all-classes_gray.csv'

trigger_type_aux_str = None
if 'confusion-matrix' in path_csv:
    print('Approach: confusion matrix and original CNN')
    X, y = read_features_confusion_matrix(path_csv, trigger_type_aux_str)
else:
    print('Approach: confusion distribution and SDNs')
    X, y = read_features(path_csv, trigger_type_aux_str) # clean data is automatically added
print(X.shape, y.shape)

search_space = list()
## LogisticRegression params
search_space.append(Real(0.001, 100.0, 'log-uniform', name='C'))

## SVM params
# search_space.append(Real(0.00001, 100.0, 'log-uniform', name='C'))
# # search_space.append(Integer(1, 5, name='degree'))
# # search_space.append(Real(0.00001, 100.0, 'log-uniform', name='gamma'))
# search_space.append(Categorical(['scale'], name='gamma'))
# # search_space.append(Categorical(['rbf'], name='kernel'))
# search_space.append(Categorical(['rbf'], name='kernel')) # linear, poly, rbf, sigmoid

@use_named_args(search_space)
def evaluate_model(**params):
    model = LogisticRegression()
    model.set_params(**params)
#     params['probability'] = True
#     model = svm.SVC()
#     model.set_params(**params)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    result = cross_val_score(model, X, y, cv=cv, n_jobs=-1, scoring='neg_log_loss')
    estimate = np.mean(result)
    return abs(estimate)

result = gp_minimize(evaluate_model, search_space)
print('Best Score: %.3f' % (result.fun))
print('Best Parameters: %s' % (result.x))

Approach: confusion distribution and SDNs
Using all rows (clean, polygon and instagram filters)
(1008, 12) (1008,)
Best Score: 0.539
Best Parameters: [1.9807706419673599]


# Local Testing: train on training data and test on holdout data

In [ ]:
path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray.csv'
# path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
# path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
trigger_type_aux_str = None

print('Local Testing')
if 'confusion-matrix' in path_train_csv and 'confusion-matrix' in path_holdout_csv:
    print('Approach: confusion matrix and original CNN')
    X_train, y_train = read_features_confusion_matrix(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features_confusion_matrix(path_holdout_csv, trigger_type_aux_str)
else:
    print('Approach: confusion distribution and SDNs')
    X_train, y_train = read_features(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features(path_holdout_csv, trigger_type_aux_str)

print('train shape:', X_train.shape, y_train.shape)
print('holdout shape:', X_holdout.shape, y_holdout.shape)

roc, xent = evaluate_classifier(X_train, y_train, X_holdout, y_holdout)
print(f'ROC AUC = {roc}')
print(f'Cross-Entropy = {xent}')

# Train meta-model using square data and filters

In [27]:
# path_csv = r'confusion-reports/ics_svm/round2-train-dataset/round2-train-dataset_square-25-filters_all-classes_gray.csv'
path_csv = r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random-filters_all-classes_gray.csv'

X_train, y_train = read_features(path_csv, None)
# model = svm.SVC(C=11, kernel='rbf', gamma='scale', probability=True)
model = LogisticRegression(C=2)
model.fit(X_train, y_train)
save_obj(model, r'..\metamodel_07_svm_round3_LR_square35_RANDOM_filters_all-classes.pickle')
print('done')

Using all rows (clean, polygon and instagram filters)
done


# Train Neural Network

In [135]:
def create_neural_network():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=12, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def cross_validation(data_x, data_y):
    pipeline = Pipeline([('mlp', KerasClassifier(build_fn=create_neural_network, epochs=100, batch_size=32, verbose=0))])
    kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=666)
    results = cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_log_loss', n_jobs=-1)
    results = results[~np.isnan(results)]
    results = results[~np.isinf(results)]
    print(f'CrossEntropy: mean={-results.mean()} std={results.std()}')

def keras_save(model, folder):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    model_json = model.to_json()
    with open(os.path.join(folder, 'model.json'), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(os.path.join(folder, 'model.h5'))

def keras_load(folder):
    from keras.models import model_from_json

    json_file = open(os.path.join(folder, 'model.json'), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(os.path.join(folder, 'model.h5'))
    return loaded_model
    
def train_nn(data_x, data_y):
    model = create_neural_network()
    history = model.fit(data_x, data_y, epochs=100, batch_size=32, verbose=0)
    
    loss, accuracy = model.evaluate(data_x, data_y, verbose=1)
    print(f'loss={loss}\naccuracy={accuracy}')
    keras_save(model, r'D:\Cloud\MEGA\TrojAI\TrojAI-UMD\metamodel_08_svm_round3_NN-60-30_square30_RANDOM_filters_all-classes')
    
path_csv = r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-30-random-filters_all-classes_gray.csv'
X, y = read_features(path_csv, None)

# cross_validation(X, y)
# train_nn(X, y)

Using all rows (clean, polygon and instagram filters)


In [136]:
MODEL = keras_load(r'..\metamodel_08_svm_round3_NN-60-30_square30_RANDOM_filters_all-classes')

In [149]:
a = MODEL.weights
a[5]

<tf.Variable 'dense_869/bias:0' shape=(1,) dtype=float32, numpy=array([-0.15591685], dtype=float32)>

# merge or simplify datasets

In [20]:
def select_columns_from_dataset():
    df = pd.read_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-25-filters_all-classes_gray.csv')
    for col_to_delete in ['square25_mean_diff', 'square25_std_diff', 'square25_mean', 'square25_std']:
        del df[col_to_delete]
    df.to_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_filters_all-classes_gray.csv', index=False)
    print('done')

def merge_datasets():
    for size in [25, 30, 35, 40, 45, 50]:
        df1 = pd.read_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_filters_all-classes_gray.csv')
        df2 = pd.read_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_squares-random-all-classes-gray.csv')
        df3 = pd.read_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-columns.csv')
        for col in [f'square{size}_r_mean_diff', f'square{size}_r_std_diff', f'square{size}_r_mean', f'square{size}_r_std']:
            df1[col] = df2[col]
        df1['trigger_type_aux'] = df3['trigger_type_aux']
        new_order_for_columns = [
            'model_name', 'model_architecture', 'model_label',
            'trigger_type_aux',

            f'square{size}_r_mean_diff', f'square{size}_r_std_diff',
            'gotham_mean_diff', 'gotham_std_diff',
            'kelvin_mean_diff', 'kelvin_std_diff',
            'lomo_mean_diff', 'lomo_std_diff',
            'nashville_mean_diff', 'nashville_std_diff',
            'toaster_mean_diff', 'toaster_std_diff',

            'clean_mean', 'clean_std',
            f'square{size}_r_mean', f'square{size}_r_std',
            'gotham_mean', 'gotham_std',
            'kelvin_mean', 'kelvin_std',
            'lomo_mean', 'lomo_std',
            'nashville_mean', 'nashville_std',
            'toaster_mean', 'toaster_std',

            'trigger_color', 'num_classes',
        ]
        df1 = df1[new_order_for_columns]
        df1.to_csv(fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}-random-filters_all-classes_gray.csv', index=False) 
        print('done')
merge_datasets()
# select_columns_from_dataset()

done
done
done
done
done
done


# create correct trigger_type_aux column

In [19]:
metadata = pd.read_csv(r'D:\Cloud\MEGA\TrojAI\TrojAI-data\round3-train-dataset\METADATA.csv')
# metadata.columns
metadata = metadata[['model_name', 'poisoned', 'model_architecture', 'trigger_type', 'instagram_filter_type', 'polygon_side_count']]
metadata['trigger_type_aux'] = [''] * len(metadata)
# set(metadata['instagram_filter_type'].to_list())
for i in range(len(metadata)):
    trigger_type = metadata['trigger_type'].iloc[i]
    instagram_filter_type = metadata['instagram_filter_type'].iloc[i].lower().replace('filter','').replace('xform', '')
    polygon_side_count = metadata['polygon_side_count'].iloc[i]
    if trigger_type == 'None':
        metadata['trigger_type_aux'].iloc[i] = 'none'
    elif trigger_type == 'instagram':
        metadata['trigger_type_aux'].iloc[i] = f'instagram-{instagram_filter_type}'
    elif trigger_type == 'polygon':
        metadata['trigger_type_aux'].iloc[i] = f'polygon-{polygon_side_count}'
metadata.to_csv(rf'D:\Cloud\MEGA\TrojAI\TrojAI-UMD\notebooks\confusion-reports\ics_svm\round3-train-dataset\round3-columns.csv', index=False)